In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from imblearn.over_sampling import SMOTE

In [47]:
def detect_missing_values(df):
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    return missing_data.sort_values(ascending=False)

def detect_outliers(df, columns):
    outliers = {}
    for column in columns:
            # Use IQR method
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outlier_rows = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
            outliers[column] = outlier_rows[[column]]
    
    return outliers

def detect_duplicates(df):
    duplicate_rows = df[df.duplicated()]
    return duplicate_rows

In [48]:
# Load the teams_post data
teams = pd.read_csv('./data/teams.csv')

# Convert 'playoff' column to binary (1 for 'Y', 0 for 'N')
teams['playoff'] = teams['playoff'].map({'Y': 1, 'N': 0})

zero_cols = ["tmORB", "tmDRB", "tmTRB", "opptmORB", "opptmDRB", "opptmTRB"]

# Since "tmORB", "tmDRB", "tmTRB", "opptmORB", "opptmDRB", and "opptmTRB" contain only zero values,
# and are redundant with "o_oreb", "o_dreb", "o_reb" for team stats, and "d_oreb", "d_dreb", "d_reb" for opponent stats,
# we drop the redundant columns.

teams = teams.drop(columns=zero_cols)
print("Dropped redundant rebound columns and renamed others for clarity.")

# Drop the 'divID' column as it contains only empty strings and does not add useful information
teams = teams.drop(columns=['divID'])
print("Dropped 'divID' column as it contains no information.")

# Drop the 'seeded' column as it contains only zero values
teams = teams.drop(columns=['seeded'])
print("Dropped 'seeded' column as it contains only zero values.")

def calculate_playoff_score(row):
    if row['finals'] == 'W':
        return 4  # Won the championship
    elif row['finals'] == 'L':
        return 3  # Lost in the finals
    elif row['semis'] == 'L':
        return 2  # Lost in the semifinals
    elif row['firstRound'] == 'L':
        return 1  # Lost in the first round
    else:
        return 0  # Did not make the playoffs

# Apply the function to each row to create the playoff_progression_score
teams['playoff_progression_score'] = teams.apply(calculate_playoff_score, axis=1)

# Drop the original 'firstRound', 'semis', and 'finals' columns as they are now redundant
teams = teams.drop(columns=['firstRound', 'semis', 'finals'])

# Drop columns that don't add predictive value
# 'lgID': Contains only "WNBA" for every row, so it provides no additional information.
# 'franchID': Redundant identifier, as 'tmID' already identifies each team uniquely.
# 'confID': Lacks value without conference-specific qualification/matchup data.
# 'name': Purely descriptive and irrelevant to playoff predictions.
# 'arena': Also descriptive and does not impact playoff qualification.
teams = teams.drop(columns=['lgID', 'franchID', 'confID', 'name', 'arena'])
print("Dropped 'lgID', 'franchID', 'confID', 'name', and 'arena' as they are irrelevant for predictive modeling.")

## DETECTION OF MISSING VALUES
#missing_values = detect_missing_values(teams)
#print("\nMissing Values")
#print(missing_values)
#
## DETECTION OF DUPLICATES
#duplicate_rows = detect_duplicates(teams)
#print("\nDuplicates")
#print(duplicate_rows)
#
## DETECTION OF OUTLIERS
## Select numerical columns only
#numeric_columns = teams.select_dtypes(include=['float64', 'int64']).columns
#
#for column in numeric_columns:
#    plt.figure(figsize=(8, 4))
#    sns.boxplot(x=teams[column])
#    plt.title(f'Box Plot of {column}')
#    plt.show()
#
#outliers = detect_outliers(teams, numeric_columns)
#
#for col, outlier_data in outliers.items():
#    print(f"Outliers in {col}:\n{outlier_data}\n")
#
## Plot for 'next_season_playoff' column
#plt.figure(figsize=(6, 4))
#plt.bar(next_playoff_counts.index, next_playoff_counts.values)
#plt.title('Data Balance in Next Season Playoff')
#plt.xlabel('Next Season Playoff (0 = No, 1 = Yes)')
#plt.ylabel('Count')
#plt.xticks([0, 1])
#plt.show()

Dropped redundant rebound columns and renamed others for clarity.
Dropped 'divID' column as it contains no information.
Dropped 'seeded' column as it contains only zero values.
Dropped 'lgID', 'franchID', 'confID', 'name', and 'arena' as they are irrelevant for predictive modeling.


In [49]:
# Load the players_teams data
players_teams = pd.read_csv("./data/players_teams.csv")

# Drop 'lgID' as it contains only "WNBA" and provides no unique value
players_teams = players_teams.drop(columns=['lgID'])

# Detect and drop duplicates
duplicates = players_teams[players_teams.duplicated()]
if not duplicates.empty:
    print("Duplicates detected. Removing duplicate rows.")
    players_teams = players_teams.drop_duplicates()
else:
    print("No duplicates found.")

# Display a sample of the dataframe to verify changes
display(players_teams.head())

# Store cleaned csv
players_teams.to_csv('./cleaned_data/players_teams.csv', index=False)

No duplicates found.


,playerID,year,stint,tmID,GP,GS,minutes,points,oRebounds,dRebounds,...,PostBlocks,PostTurnovers,PostPF,PostfgAttempted,PostfgMade,PostftAttempted,PostftMade,PostthreeAttempted,PostthreeMade,PostDQ
0,abrossv01w,2,0,MIN,26.0,23.0,846.0,343.0,43.0,131.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abrossv01w,3,0,MIN,27.0,27.0,805.0,314.0,45.0,101.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,abrossv01w,4,0,MIN,30.0,25.0,792.0,318.0,44.0,97.0,...,1.0,8.0,8.0,22.0,6.0,8.0,8.0,7.0,3.0,0.0
3,abrossv01w,5,0,MIN,22.0,11.0,462.0,146.0,17.0,57.0,...,2.0,3.0,7.0,23.0,8.0,4.0,2.0,8.0,2.0,0.0
4,abrossv01w,6,0,MIN,31.0,31.0,777.0,304.0,29.0,78.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Load the players data
players = pd.read_csv("./data/players.csv")

# Filter out rows in players that do not have corresponding playerIDs in players_teams
valid_player_ids = players_teams['playerID'].unique()
players = players[players['bioID'].isin(valid_player_ids)]

# Show that all values in firstseason and lastseason are '0'
firstseason_all_zero = (players['firstseason'] == 0).all()
lastseason_all_zero = (players['lastseason'] == 0).all()

print("All values in 'firstseason' are 0:", firstseason_all_zero)
print("All values in 'lastseason' are 0:", lastseason_all_zero)

# Show the only valida player with a registered Death Date
non_zero_death_dates = players[players['deathDate'] != "0000-00-00"]
display(non_zero_death_dates.head())

# Even though there is 1 registered Death Date, it really doesn't add anything. Birth Date is kept, for potential aging information.
players = players.drop(columns=['firstseason', 'lastseason', 'deathDate'])
print("Dropped 'firstseason', 'lastseason', and 'deathDate' columns as they contain only irrelevant values.")

# Detect and drop duplicates
duplicates = players[players.duplicated()]
if not duplicates.empty:
    print("Duplicates detected. Removing duplicate rows.")
    players = players.drop_duplicates()
else:
    print("No duplicates found.")

# Display a sample of the dataframe to verify changes
display(players.head())

# Store cleaned csv
players.to_csv('./cleaned_data/players.csv', index=False)

All values in 'firstseason' are 0: True
All values in 'lastseason' are 0: True


,bioID,pos,firstseason,lastseason,height,weight,college,collegeOther,birthDate,deathDate
225,dydekma01w,C,0,0,9.0,223,NaN,NaN,1974-04-28,2011-05-27


Dropped 'firstseason', 'lastseason', and 'deathDate' columns as they contain only irrelevant values.
No duplicates found.


,bioID,pos,height,weight,college,collegeOther,birthDate
1,abrossv01w,F,74.0,169,Connecticut,NaN,1980-07-09
2,adairje01w,C,76.0,197,George Washington,NaN,1986-12-19
3,adamsda01w,F-C,73.0,239,Texas A&M,Jefferson College (JC),1989-02-19
4,adamsjo01w,C,75.0,180,New Mexico,NaN,1981-05-24
8,aguilel01w,G,67.0,165,George Washington,NaN,1976-10-15


In [51]:
# Load the awards_players data
awards_players = pd.read_csv("./data/awards_players.csv")

# Drop 'lgID' column as it provides no unique value
awards_players = awards_players.drop(columns=['lgID'])

# Separate dataframes for player awards and coach awards
player_awards = awards_players[~awards_players['award'].str.contains("Coach")].copy()
coach_awards = awards_players[awards_players['award'].str.contains("Coach")].copy()

# Standardize award names
award_name_mapping = {
    "Kim Perrot Sportsmanship": "Kim Perrot Sportsmanship Award",
    "Kim Perrot Sportsmanship Award": "Kim Perrot Sportsmanship Award",
    "All-Star Game Most Valuable Player": "All-Star Game MVP",
    "Most Valuable Player": "MVP",
    "WNBA Finals Most Valuable Player": "Finals MVP",
    "Sixth Woman of the Year": "6th Woman of the Year",
    "WNBA All-Decade Team": "All-Decade Team",
    "WNBA All Decade Team Honorable Mention": "All-Decade Team Honorable Mention"
}

player_awards.loc[:, 'award'] = player_awards['award'].map(award_name_mapping).fillna(player_awards['award'])
coach_awards.loc[:, 'award'] = coach_awards['award'].map(award_name_mapping).fillna(coach_awards['award'])

# Detect and drop duplicates
duplicates_player = player_awards[player_awards.duplicated()]
if not duplicates_player.empty:
    print("Duplicates detected in player awards. Removing duplicate rows.")
    player_awards = player_awards.drop_duplicates()
else:
    print("No duplicates found in player awards.")

duplicates_coach = coach_awards[coach_awards.duplicated()]
if not duplicates_coach.empty:
    print("Duplicates detected in coach awards. Removing duplicate rows.")
    coach_awards = coach_awards.drop_duplicates()
else:
    print("No duplicates found in coach awards.")

# Swap 'playerID' for 'coachID' in coach awards
coach_awards = coach_awards.rename(columns={'playerID': 'coachID'})

# Display samples of both dataframes to verify transformations
display(player_awards.head())
display(coach_awards.head())

# Store cleaned csvs
player_awards.to_csv('./cleaned_data/player_awards.csv', index=False)
coach_awards.to_csv('./cleaned_data/coach_awards.csv', index=False)

No duplicates found in player awards.
No duplicates found in coach awards.


,playerID,award,year
0,thompti01w,All-Star Game MVP,1
1,leslili01w,All-Star Game MVP,2
2,leslili01w,All-Star Game MVP,3
3,teaslni01w,All-Star Game MVP,4
4,swoopsh01w,All-Star Game MVP,6


,coachID,award,year
8,coopemi01w,Coach of the Year,1
9,hugheda99w,Coach of the Year,2
10,stanlma99w,Coach of the Year,3
11,laimbbi01w,Coach of the Year,4
12,mcconsu01w,Coach of the Year,5


In [52]:
# Load the coaches data
coaches = pd.read_csv("./data/coaches.csv")

# Drop 'lgID' as it is only "WNBA" and provides no unique value
coaches = coaches.drop(columns=['lgID'])

# Detect and drop duplicates
duplicates = coaches[coaches.duplicated()]
if not duplicates.empty:
    print("Duplicates detected. Removing duplicate rows.")
    coaches = coaches.drop_duplicates()
else:
    print("No duplicates found.")

# Display a sample of the dataframe to verify changes
display(coaches.head())

coaches.to_csv('./cleaned_data/coaches.csv', index=False)

No duplicates found.


,coachID,year,tmID,stint,won,lost,post_wins,post_losses
0,adamsmi01w,5,WAS,0,17.0,17.0,1.0,2.0
1,adubari99w,1,NYL,0,20.0,12.0,4.0,3.0
2,adubari99w,2,NYL,0,21.0,11.0,3.0,3.0
3,adubari99w,3,NYL,0,18.0,14.0,4.0,4.0
4,adubari99w,4,NYL,0,16.0,18.0,0.0,0.0


In [53]:
# Calculate statistics for the new dataframe
dataset = pd.DataFrame()

# Team Statistics
dataset['Playoff'] = teams['playoff']
dataset['Team'] = teams['tmID']
dataset['Year'] = teams['year']
dataset['Rank'] = teams['rank']
dataset['PlayoffProgScore'] = teams['playoff_progression_score']
dataset['GP'] = teams['GP']
dataset['W'] = teams['won']
dataset['L'] = teams['lost']
dataset['WIN%'] = 100 * (teams['won'] / teams['GP'])
dataset['MIN'] = teams['min']
dataset['PTS'] = teams['o_pts']
dataset['FGM'] = teams['o_fgm']
dataset['FGA'] = teams['o_fga']
dataset['FG%'] = 100 * (teams['o_fgm'] / teams['o_fga'])
dataset['3PM'] = teams['o_3pm']
dataset['3PA'] = teams['o_3pa']
dataset['3P%'] = 100 * (teams['o_3pm'] / teams['o_3pa'])
dataset['FTM'] = teams['o_ftm']
dataset['FTA'] = teams['o_fta']
dataset['FT%'] = 100 * (teams['o_ftm'] / teams['o_fta'])
dataset['OREB'] = teams['o_oreb']
dataset['DREB'] = teams['o_dreb']
dataset['REB'] = teams['o_reb']
dataset['AST'] = teams['o_asts']
dataset['TOV'] = teams['o_to']
dataset['STL'] = teams['o_stl']
dataset['BLK'] = teams['o_blk']
dataset['BLKA'] = teams['d_blk']
dataset['PF'] = teams['o_pf']
dataset['PFD'] = teams['d_pf']

# Advanced
dataset['POSS'] = 0.5 * (
    (teams['o_fga'] + 0.4 * teams['o_fta'] -
     1.07 * (teams['o_oreb'] / (teams['o_oreb'] + teams['d_dreb'])) *
     (teams['o_fga'] - teams['o_fgm']) + teams['o_to']) +
    (teams['d_fga'] + 0.4 * teams['d_fta'] -
     1.07 * (teams['d_oreb'] / (teams['d_oreb'] + teams['o_dreb'])) *
     (teams['d_fga'] - teams['d_fgm']) + teams['d_to'])
)
dataset['OFFRTG'] = 100 * (teams['o_pts'] / dataset['POSS'])
dataset['DEFRTG'] = 100 * (teams['d_pts'] / dataset['POSS'])
dataset['NETRTG'] = dataset['OFFRTG'] - dataset['DEFRTG']
dataset['AST/TO'] = teams['o_asts'] / teams['o_to']
dataset['AST RATIO'] = (teams['o_asts'] * 100) / dataset['POSS']
dataset['OREB%'] = (
    100 * (teams['o_oreb'] * (dataset['MIN'] / 5)) / 
    (dataset['MIN'] * (teams['o_oreb'] + teams['d_dreb']))
)
dataset['DREB%'] = (
    100 * (teams['o_dreb'] * (dataset['MIN'] / 5)) / 
    (dataset['MIN'] * (teams['o_dreb'] + teams['d_oreb']))
)
dataset['REB%'] = (
    100 * (teams['o_reb'] * (dataset['MIN'] / 5)) / 
    (dataset['MIN'] * (teams['o_reb'] + teams['d_reb']))
)
dataset['TOV%'] = 100 * teams['o_to'] / (
    teams['o_fga'] + 0.44 * teams['o_fta'] + teams['o_to']
)
dataset['EFG%'] = 100 * ((teams['o_fgm'] + (0.5 * teams['o_3pm'])) / teams['o_fga'])
dataset['TS%'] = 100 * (teams['o_pts'] / (2 * (teams['o_fga'] + 0.44 * teams['o_fta'])))

OPPPOSS = 0.5 * (
    (teams['d_fga'] + 0.4 * teams['d_fta'] -
     1.07 * (teams['d_oreb'] / (teams['d_oreb'] + teams['o_dreb'])) *
     (teams['d_fga'] - teams['d_fgm']) + teams['d_to']) +
    (teams['o_fga'] + 0.4 * teams['o_fta'] -
     1.07 * (teams['o_oreb'] / (teams['o_oreb'] + teams['d_dreb'])) *
     (teams['o_fga'] - teams['o_fgm']) + teams['o_to'])
)
dataset['PACE'] = 40 * ((dataset['POSS'] + OPPPOSS) / (2 * (dataset['MIN'] / 5)))

# Rename player stats columns to match team stats naming convention
players_teams = players_teams.rename(columns={
    'points': 'PTS',
    'fgMade': 'FGM',
    'ftMade': 'FTM',
    'fgAttempted': 'FGA',
    'ftAttempted': 'FTA',
    'dRebounds': 'DREB',
    'oRebounds': 'OREB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'PF': 'PF',
    'turnovers': 'TOV'
})

# Merge player and team datasets on team ID and year
merged_data = players_teams.merge(
    dataset,
    left_on=['tmID', 'year'],
    right_on=['Team', 'Year'],
    suffixes=('', '_team')
)

# Calculate GmStats as a proportion of team stats
merged_data['games_ratio'] = merged_data['GP'] / merged_data['GP_team']

# Define game-level stats for players
team_stat_columns = [
    'PTS', 'FGM', 'FTM', 'FGA', 'FTA', 'DREB', 'OREB', 'AST', 'STL', 'BLK', 'PF', 'TOV'
]

for stat in team_stat_columns:
    merged_data[f'Gm{stat}'] = merged_data['games_ratio'] * merged_data[f'{stat}_team']

# Calculate PIE for each player
merged_data['PIE'] = (
    merged_data['PTS'] + 
    merged_data['FGM'] + 
    merged_data['FTM'] - 
    merged_data['FGA'] - 
    merged_data['FTA'] + 
    merged_data['DREB'] + 
    (0.5 * merged_data['OREB']) + 
    merged_data['AST'] + 
    merged_data['STL'] + 
    (0.5 * merged_data['BLK']) - 
    merged_data['PF'] - 
    merged_data['TOV']
) / (
    merged_data['GmPTS'] + 
    merged_data['GmFGM'] + 
    merged_data['GmFTM'] - 
    merged_data['GmFGA'] - 
    merged_data['GmFTA'] + 
    merged_data['GmDREB'] + 
    (0.5 * merged_data['GmOREB']) + 
    merged_data['GmAST'] + 
    merged_data['GmSTL'] + 
    (0.5 * merged_data['GmBLK']) - 
    merged_data['GmPF'] - 
    merged_data['GmTOV']
)

# Average PIE for players with multiple stints in the same year
merged_data = merged_data.groupby(['playerID', 'year'], as_index=False).agg({
    'PIE': 'mean',
    'tmID': 'first',  # Retain the first team ID for simplicity
    'games_ratio': 'sum'  # Sum games ratio to ensure accurate contribution
})

coaches_cer = pd.read_csv('./cleaned_data/coaches_ema.csv')

# Step 1: Ensure unique team-year combinations in 'dataset'
dataset_unique = dataset.drop_duplicates(subset=['Team', 'Year'])

# Step 2: Create dictionaries for quick access
pie_dict = merged_data.set_index(['playerID', 'year'])['PIE'].to_dict()
coach_cer_dict = coaches_cer.set_index(['coachID', 'year'])['CER'].to_dict()
coach_dict = coaches.set_index(['tmID', 'year'])['coachID'].to_dict()

# Create dictionaries of all years available for coaches and players
coach_years = {}
for coach_id, year in coach_cer_dict.keys():
    if coach_id not in coach_years:
        coach_years[coach_id] = []
    coach_years[coach_id].append(year)

player_years = {}
for pid, year in pie_dict.keys():
    if pid not in player_years:
        player_years[pid] = []
    player_years[pid].append(year)

# Calculate yearly averages
yearly_avg_cer = coaches_cer.groupby('year')['CER'].mean().to_dict()
yearly_avg_pie = merged_data.groupby('year')['PIE'].mean().to_dict()

# Initialize new columns with None
dataset['AvgPIE_NextYearPlayers'] = None
dataset['CER_NextYearCoach'] = None

# Process each team-year combination
for index, row in dataset.iterrows():
    team = row['Team']
    year = row['Year']
    next_year = year + 1
    
    # Get next year's coach and their CER
    next_year_coach = coach_dict.get((team, next_year))
    current_year_cer = None
    
    if next_year_coach:
        # Try to get current year CER
        current_year_cer = coach_cer_dict.get((next_year_coach, year))
        
        if current_year_cer is None and next_year_coach in coach_years:
            # If no current year CER, find the most recent previous CER
            previous_years = [y for y in coach_years[next_year_coach] if y < year]
            if previous_years:
                most_recent_year = max(previous_years)
                current_year_cer = coach_cer_dict.get((next_year_coach, most_recent_year))
        
        if current_year_cer is None:
            # If still no CER (rookie coach), use year average
            current_year_cer = yearly_avg_cer.get(year)
    
    # Get next year's players and their PIE values
    players_next_year = players_teams[
        (players_teams['tmID'] == team) & (players_teams['year'] == next_year)
    ]
    
    avg_pie = None
    if not players_next_year.empty:
        player_ids = players_next_year['playerID'].unique()
        team_pies = []
        
        for pid in player_ids:
            pie = None
            
            # Try to get current year PIE
            pie_key = (pid, year)
            pie = pie_dict.get(pie_key)
            
            if pie is None and pid in player_years:
                # If no current year PIE, find the most recent previous PIE
                previous_years = [y for y in player_years[pid] if y < year]
                if previous_years:
                    most_recent_year = max(previous_years)
                    pie = pie_dict.get((pid, most_recent_year))
            
            if pie is None:
                # If still no PIE (rookie), use year average
                pie = yearly_avg_pie.get(year)
            
            if pie is not None:
                team_pies.append(pie)
        
        if team_pies:
            avg_pie = sum(team_pies) / len(team_pies)
    
    # Update the dataset
    dataset.loc[index, 'AvgPIE_NextYearPlayers'] = avg_pie
    dataset.loc[index, 'CER_NextYearCoach'] = current_year_cer

# Convert columns to float type for consistency
dataset['AvgPIE_NextYearPlayers'] = dataset['AvgPIE_NextYearPlayers'].astype(float)
dataset['CER_NextYearCoach'] = dataset['CER_NextYearCoach'].astype(float)

# First get the earliest year for each team to identify rookie seasons
team_first_years = dataset.groupby('Team')['Year'].min()
rookie_teams_data = dataset[dataset.apply(lambda x: x['Year'] == team_first_years[x['Team']], axis=1)]

# Calculate average values for rookie teams
rookie_averages = rookie_teams_data.mean(numeric_only=True)

# Create new row for TUL
new_team_row = pd.Series(rookie_averages)
new_team_row['Team'] = 'TUL'
new_team_row['Year'] = 10

# Add the new row to the dataset
dataset = pd.concat([dataset, pd.DataFrame([new_team_row])], ignore_index=True)

# Sort dataset by Team and Year for clarity
dataset = dataset.sort_values(['Team', 'Year'])

display(dataset.head())

# Filter the dataset for year 10
year_10_data = dataset[dataset['Year'] == 10]

# Save the filtered data to a CSV file
output_file_path = './cleaned_data/year_10_stats.csv'  # Modify the path if needed
year_10_data.to_csv(output_file_path, index=False)

print(f"Year 10 stats saved to {output_file_path}")


,Playoff,Team,Year,Rank,PlayoffProgScore,GP,W,L,WIN%,MIN,...,AST RATIO,OREB%,DREB%,REB%,TOV%,EFG%,TS%,PACE,AvgPIE_NextYearPlayers,CER_NextYearCoach
0,0.0,ATL,9,7.0,0.0,34.0,4.0,30.0,11.764706,6825.0,...,18.075514,5.647841,12.952548,9.197267,18.706625,44.109832,49.165697,79.763058,0.118533,0.040000
1,1.0,ATL,10,2.0,1.0,34.0,18.0,16.0,52.941176,6950.0,...,19.169890,6.595918,14.155629,10.349363,17.610889,47.199341,51.825955,82.113185,0.104153,0.490000
2,0.0,CHA,1,8.0,0.0,32.0,8.0,24.0,25.000000,6475.0,...,24.373637,6.295150,13.179916,9.714286,18.696662,46.111403,50.675049,69.826695,0.102416,0.090000
3,1.0,CHA,2,4.0,3.0,32.0,18.0,14.0,56.250000,6500.0,...,22.368024,7.054795,13.001017,10.199032,19.064320,46.207865,51.060468,64.240053,0.085999,0.320000
4,1.0,CHA,3,2.0,1.0,32.0,18.0,14.0,56.250000,6450.0,...,23.433293,6.550976,13.776371,10.213904,16.387385,48.910615,53.825683,65.632452,0.101880,0.238421


Year 10 stats saved to ./cleaned_data/year_10_stats.csv


In [54]:
# Label
teams = teams.sort_values(by=['tmID', 'year']).reset_index(drop=True)
dataset['PlayoffNextSeason'] = teams.groupby('tmID')['playoff'].shift(-1)
dataset = dataset.dropna(subset=['PlayoffNextSeason'])
dataset['PlayoffNextSeason'] = dataset['PlayoffNextSeason'].astype(int)

In [55]:
dataset = dataset.drop(columns=['Team', 'Year'])

# Separate features and target variable
X = dataset.drop(columns=['PlayoffNextSeason'])
y = dataset['PlayoffNextSeason']

display(y.value_counts())

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verify the new class distribution after applying SMOTE
display(y_resampled.value_counts())

# Merge resampled data into a new DataFrame
balanced_dataset = pd.DataFrame(X_resampled, columns=X.columns)
balanced_dataset['PlayoffNextSeason'] = y_resampled

dataset = balanced_dataset

PlayoffNextSeason
1    71
0    51
Name: count, dtype: int64

PlayoffNextSeason
1    71
0    71
Name: count, dtype: int64

In [56]:
# Save the new dataset to a CSV file
output_file_path = './cleaned_data/dataset.csv'
dataset.to_csv(output_file_path, index=False)

print(f"Processed dataset saved to {output_file_path}")

Processed dataset saved to ./cleaned_data/dataset.csv


In [57]:
'''
# Load the teams_post data
teams_post = pd.read_csv("./data/teams_post.csv")

# Drop 'lgID' column as it contains only "WNBA" for every row
teams_post = teams_post.drop(columns=['lgID'])

# Detect and drop duplicates
duplicates = teams_post[teams_post.duplicated()]
if not duplicates.empty:
    print("Duplicates detected. Removing duplicate rows.")
    teams_post = teams_post.drop_duplicates()
else:
    print("No duplicates found.")

# Display a sample of the dataframe to verify changes
display(teams_post.head())

# Store cleaned csv
teams_post.to_csv('./cleaned_data/teams_post.csv', index=False)
'''

'\n# Load the teams_post data\nteams_post = pd.read_csv("./data/teams_post.csv")\n\n# Drop \'lgID\' column as it contains only "WNBA" for every row\nteams_post = teams_post.drop(columns=[\'lgID\'])\n\n# Detect and drop duplicates\nduplicates = teams_post[teams_post.duplicated()]\nif not duplicates.empty:\n    print("Duplicates detected. Removing duplicate rows.")\n    teams_post = teams_post.drop_duplicates()\nelse:\n    print("No duplicates found.")\n\n# Display a sample of the dataframe to verify changes\ndisplay(teams_post.head())\n\n# Store cleaned csv\nteams_post.to_csv(\'./cleaned_data/teams_post.csv\', index=False)\n'

In [58]:
'''
# Load the series_post data
series_post = pd.read_csv("./data/series_post.csv")

# Drop 'lgIDWinner' and 'lgIDLoser' columns as they contain only "WNBA" and add no value
series_post = series_post.drop(columns=['lgIDWinner', 'lgIDLoser'])

# Detect and drop duplicates
duplicates = series_post[series_post.duplicated()]
if not duplicates.empty:
    print("Duplicates detected. Removing duplicate rows.")
    series_post = series_post.drop_duplicates()
else:
    print("No duplicates found.")

# Display a sample of the dataframe to verify changes
display(series_post.head())

# Store cleaned csv
series_post.to_csv('./cleaned_data/series_post.csv', index=False)
'''

'\n# Load the series_post data\nseries_post = pd.read_csv("./data/series_post.csv")\n\n# Drop \'lgIDWinner\' and \'lgIDLoser\' columns as they contain only "WNBA" and add no value\nseries_post = series_post.drop(columns=[\'lgIDWinner\', \'lgIDLoser\'])\n\n# Detect and drop duplicates\nduplicates = series_post[series_post.duplicated()]\nif not duplicates.empty:\n    print("Duplicates detected. Removing duplicate rows.")\n    series_post = series_post.drop_duplicates()\nelse:\n    print("No duplicates found.")\n\n# Display a sample of the dataframe to verify changes\ndisplay(series_post.head())\n\n# Store cleaned csv\nseries_post.to_csv(\'./cleaned_data/series_post.csv\', index=False)\n'

In [59]:
'''
# Aggregate data by coachID and year (ignoring stint and tmID)
coaches_agg = coaches.groupby(['coachID', 'year']).agg({
    'won': 'sum',
    'lost': 'sum',
    'post_wins': 'sum',
    'post_losses': 'sum'
}).reset_index()

# Preview the aggregated data
print(coaches_agg.head())
'''

"\n# Aggregate data by coachID and year (ignoring stint and tmID)\ncoaches_agg = coaches.groupby(['coachID', 'year']).agg({\n    'won': 'sum',\n    'lost': 'sum',\n    'post_wins': 'sum',\n    'post_losses': 'sum'\n}).reset_index()\n\n# Preview the aggregated data\nprint(coaches_agg.head())\n"

In [60]:
'''
# Find the coach with the maximum post_wins for each year
max_post_wins_per_year = coaches_agg.loc[coaches_agg.groupby('year')['post_wins'].idxmax()][['year', 'coachID', 'post_wins']]
max_post_wins_per_year = max_post_wins_per_year.rename(columns={'post_wins': 'max_post_wins'})

# Merge this information back with the original DataFrame
coaches_agg = coaches_agg.merge(max_post_wins_per_year, on=['year', 'coachID'], how='left')

# Set the championship indicator
coaches_agg['championship'] = 0
coaches_agg.loc[coaches_agg.set_index(['year', 'coachID']).index.isin(max_post_wins_per_year.set_index(['year', 'coachID']).index) & (coaches_agg['post_wins'] >= 6), 'championship'] = 1

# Drop the temporary max_post_wins column
coaches_agg.drop(columns=['max_post_wins'], inplace=True)

# Recalculate metrics
coaches_agg['win_ratio'] = (coaches_agg['won'] / (coaches_agg['won'] + coaches_agg['lost'])).round(4)
coaches_agg['post_win_ratio'] = (coaches_agg['post_wins'] / (coaches_agg['post_wins'] + coaches_agg['post_losses'])).round(4)

# If a coach has not reached playoffs, post_win% will be NaN. Fill these with 0.
coaches_agg['post_win_ratio'] = coaches_agg['post_win_ratio'].fillna(0)

# Merge with Awards Data for Coach of the Year
coaches_agg = coaches_agg.merge(
    coach_awards[['coachID', 'year', 'award']], 
    on=['coachID', 'year'], 
    how='left'
)
coaches_agg['COTY'] = np.where(coaches_agg['award'] == 'Coach of the Year', 1, 0)
coaches_agg.drop(columns=['award', 'won', 'lost'], inplace=True)

#Normalize features if needed
coaches_agg['win_ratio'] = coaches_agg['win_ratio'].clip(0, 1)  # Ensure the ratio is between 0 and 1
coaches_agg['post_win_ratio'] = coaches_agg['post_win_ratio'].clip(0, 1)  # Ensure the ratio is between 0 and 1

# Drop the temporary columns
coaches_agg.drop(columns=['post_wins', 'post_losses'], inplace=True)

# Preview updated data
print(coaches_agg.head())

# Store cleaned csv
coaches_agg.to_csv('./cleaned_data/coaches_agg.csv', index=False)
'''

"\n# Find the coach with the maximum post_wins for each year\nmax_post_wins_per_year = coaches_agg.loc[coaches_agg.groupby('year')['post_wins'].idxmax()][['year', 'coachID', 'post_wins']]\nmax_post_wins_per_year = max_post_wins_per_year.rename(columns={'post_wins': 'max_post_wins'})\n\n# Merge this information back with the original DataFrame\ncoaches_agg = coaches_agg.merge(max_post_wins_per_year, on=['year', 'coachID'], how='left')\n\n# Set the championship indicator\ncoaches_agg['championship'] = 0\ncoaches_agg.loc[coaches_agg.set_index(['year', 'coachID']).index.isin(max_post_wins_per_year.set_index(['year', 'coachID']).index) & (coaches_agg['post_wins'] >= 6), 'championship'] = 1\n\n# Drop the temporary max_post_wins column\ncoaches_agg.drop(columns=['max_post_wins'], inplace=True)\n\n# Recalculate metrics\ncoaches_agg['win_ratio'] = (coaches_agg['won'] / (coaches_agg['won'] + coaches_agg['lost'])).round(4)\ncoaches_agg['post_win_ratio'] = (coaches_agg['post_wins'] / (coaches_agg[